In [1]:
import jax.numpy as jnp
import gymnasium as gym
import jax.random as jrandom
from stable_baselines3 import PPO
from utils_v2 import KalmanFilter,KalmanRLWrapper

In [2]:
delta_t = .1
m=.1
M=1
g=9.8
l=.5
A=jnp.array([[0,1,0,0],
             [0,0,m*g*delta_t/M,0],
             [0,0,0,1],
             [0,0,(m+M)*g*delta_t/(M*l),0]])

#A=jnp.array([[1,delta_t,0,0],
#             [0,1,m*g*delta_t/M,0],
#             [0,0,1,delta_t],
#             [0,0,(m+M)*g*delta_t/(M*l),1]])

B=jnp.array([[0],[delta_t/M],[0],[delta_t/(M*l)]])

H=jnp.array([[0,1,0,0],
             [0,0,0,1]])

R=jnp.eye(H.shape[0])*0.005
C=H
Q=jnp.eye(A.shape[0])*0.005
mean=0
std_dev=0.005
#Q=jrandom.normal(key, shape=(A.shape[0],1)) * std_dev + mean     #FAILED
P_0=jnp.ones(A.shape)
key=jrandom.PRNGKey(42)
Z=jrandom.normal(key, shape=(C.shape[0],1)) * std_dev + mean
w_k=jnp.ones((A.shape[0],1))*0.005
#w_k=jrandom.normal(key, shape=(A.shape[0],1)) * std_dev + mean    #FAILED

In [3]:
env = gym.make("CartPole-v1")
env._max_episode_steps=1000
kf = KalmanFilter(env.reset()[0],A,B,H,R,C,Q,Z,w_k,P_0)
kalman_env = KalmanRLWrapper(env, kf)
model = PPO("MlpPolicy", kalman_env, verbose=1)
model.learn(total_timesteps=500_000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\hamed\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.3     |
|    ep_rew_mean     | 22.3     |
| time/              |          |
|    fps             | 226      |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 2048     |
---------------------------------


KeyboardInterrupt: 

In [5]:
model=PPO.load("ppo_cartpole_100k_noKF")
env = gym.make("CartPole-v1")
env._max_episode_steps=1000
rewardss = 0
episodes = 10
for ep in range(episodes):
    state = env.reset()[0]
    rewards = 0
    while True:
        action, _ = model.predict(state)
        state, reward, done, truncuated, info = env.step(action)
        rewards += reward
        if done or truncuated:
            break
    print(f"Episode: {ep+1}, Rewards: {rewards}")
    rewardss += rewards
mean=round(rewardss/(episodes),2)
print(f"mean_rewards: {mean} ({round(mean/env._max_episode_steps*100,2)} %)")

Episode: 1, Rewards: 1000.0
Episode: 2, Rewards: 1000.0
Episode: 3, Rewards: 1000.0
Episode: 4, Rewards: 1000.0
Episode: 5, Rewards: 1000.0
Episode: 6, Rewards: 1000.0
Episode: 7, Rewards: 1000.0
Episode: 8, Rewards: 1000.0
Episode: 9, Rewards: 1000.0
Episode: 10, Rewards: 1000.0
mean_rewards: 1000.0 (100.0 %)


In [26]:
model1=PPO.load("ppo_cartpole_500k")
env = gym.make("CartPole-v1")
env._max_episode_steps=1000
kf = KalmanFilter(env.reset()[0],A,B,H,R,C,Q,Z,w_k,P_0)
kalman_env = KalmanRLWrapper(env, kf)
P=P_0
rewardss = 0
episodes = 10
for ep in range(episodes):
    state_p = jnp.expand_dims(kalman_env.reset()[0], axis=-1)
    rewards = 0
    while True:
        action, _ = model1.predict(state_p.reshape(1,4).squeeze())
        action2= -1 if action == 0 else 1
        u_k = jnp.array([[action2]])
        state_p=step_estimation(A,B,state_p,u_k,w_k)
        P=process_covariance(A,P,Q)
        
        state_m, reward, done, truncuated, info = kalman_env.step(action)
        K=kalman_function(P,H,R)
        state_p,P=current_state_and_process(K,H,C,state_m,state_p,P,Z)
        rewards += reward
        if done or truncuated:
            break
    print(f"Episode: {ep+1}, Rewards: {rewards}")
    rewardss += rewards
mean=round(rewardss/(episodes),2)
print(f"mean_rewards: {mean} ({round(mean/env._max_episode_steps*100,2)} %)")

Episode: 1, Rewards: 885.0
Episode: 2, Rewards: 1000.0
Episode: 3, Rewards: 1000.0
Episode: 4, Rewards: 1000.0
Episode: 5, Rewards: 1000.0
Episode: 6, Rewards: 1000.0
Episode: 7, Rewards: 1000.0
Episode: 8, Rewards: 1000.0
Episode: 9, Rewards: 1000.0
Episode: 10, Rewards: 1000.0
mean_rewards: 988.5 (98.85 %)


In [ ]:
model1=PPO.load("ppo_cartpole_500k")
env = gym.make("CartPole-v1")
env._max_episode_steps=1000

rewardss = 0
episodes = 10
for ep in range(episodes):
    kf = KalmanFilter(env.reset()[0],A,B,H,R,C,Q,Z,w_k,P_0)
    kalman_env = KalmanRLWrapper(env, kf)
    rewards = 0
    while True:
        action, _ = model1.predict(kalman_env.kf.x_k.squeeze())
        action2= -1 if action == 0 else 1
        u_k = jnp.array([[action2]])
        #kf.predict(u_k)
        kalman_env.kf.predict(u_k)
        state_m, reward, done, truncuated, info = kalman_env.step(action)
        #kf.update(state_m)
        kalman_env.kf.update(state_m)
        rewards += reward
        if done or truncuated:
            break
    print(f"Episode: {ep+1}, Rewards: {rewards}")
    rewardss += rewards
mean=round(rewardss/(episodes),2)
print(f"mean_rewards: {mean} ({round(mean/env._max_episode_steps*100,2)} %)")

Episode: 1, Rewards: 12.0
Episode: 2, Rewards: 11.0
Episode: 3, Rewards: 12.0
Episode: 4, Rewards: 13.0
Episode: 5, Rewards: 12.0
Episode: 6, Rewards: 10.0
Episode: 7, Rewards: 12.0
Episode: 8, Rewards: 11.0
Episode: 9, Rewards: 12.0
Episode: 10, Rewards: 11.0
mean_rewards: 11.6 (1.16 %)
